<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#predicting-states-beyond-t+1" data-toc-modified-id="predicting-states-beyond-t+1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>predicting states beyond t+1</a></span><ul class="toc-item"><li><span><a href="#&quot;Temporal-Unfolding&quot;" data-toc-modified-id="&quot;Temporal-Unfolding&quot;-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>"Temporal Unfolding"</a></span></li></ul></li></ul></div>

In [2]:
for x in range(1):
    print(5)

5


# predicting states beyond t+1
The scalar predictors I've worked through so far predict outputs at "t+1". They predict 'what comes immediately next' and return one time-state to do so.

I'm awfully curious about returning t+2, t+3, so on, because:
- It should be possible, since we have TM looking at t-1, t-... -> t-cells_per_column
- It's been done! Intelletic looks 15 'bars' into the future, as per their product description, and I interpreted this as 'looking 15 states into the future'.
    - this makes a lot of sense: you don't just want the stock price t+1, or the next 5 minutes from now. You want a _range_ and _probability_, which the anomaly-categorization model does quite nicely.
- Our brains don't just look at one timestate in the future. Or do they? See below post.
    - When you see 'football incoming towards my face', your brain outputs 'collision imminent' and associates it with 'pain', which kicks off 'avoid football' routine.
        - is this your brain predicting several states (football approaching, impact, pain) after t=0? or running several consecutive calculations, each taking the former output as the new input?
        - something tells me it's the second

So how exactly is it done? I stumbled on a thread on the forums detailing one example (https://discourse.numenta.org/t/how-temporal-pooler-make-t-n-predictions-if-his-operate-only-t-t-1-timeframes/7826/2).

"""
the TM algorithm alone is only able to make a prediction one time step into the future. This is to match how the connections work in biology. But to make HTM useful in practical ML applications, a lot of non-biological components are thrown into the mix. One of these is the ability to predict further into the future that t+1.

There are a few different ways to implement this. One way is to hold a table of running averages on each cell for activity at “t+n” after that cell was active (where “n” is determined ahead of time). Since each cell is part of a high-order context, this works quite well (you are NOT asking “what is most likely to happen 25 timesteps after note G”, but something much more specific like "what is most likely to happen 25 timesteps after the 357th note of Bethoven’s 5th.
"""

- 'table of running averages on each cell', contains:
    - 'activity at t+n after that cell was active' - n is pre-determined
- cell is part of higher order context... because it's a rolling input feed, i suppose
## "Temporal Unfolding"
Alternative way:

"One possible way to forecast beyond t+1 is to make the assumption that the t+1 prediction is right and feed it back into the system, just to see what it will predict at t+2. Theoretically you could repeat the process indefinitely although errors will grow the farther you go."

"
Incidentally, this is something we usually refer to as “Temporal Unfolding”. It will most likely become a big part of HTM when we get around to performing actions (for example, combination of distal and apical input could be a mechanism for activating predictions). Dealing with uncertainty and error correction will also be a big part of temporal unfolding.
"

so if I've got this correct, if we want to predict power consumption 15 hours in the future:
- predict power(t+1), g
    - for i in range(15):
        - feed power(t+2+i) as input, add output to list
        - store this list somewhere, since we want to return it when querying "what are the 15 states after power(t)?"
- considerations:
    - when do we want to predictor.infer() these extra 14 t-states?
        - live, as in "for every piece of data, return 15 predictions"?
        - post, as in "once all the data's been eaten, re-loop through and get 14 more for each"?
            - in this case, do we keep (learn=False) for the post-extension-predictions?... probably!

I feel like you'd want to do it live. Besides, when you've got the HTM deployed and data trickles in, you'd want to do it live or else you'd have to batch every so often which would get messy.